In [1]:
import numpy as np
import os
import pandas as pd
import string as str
import math
import sys
import time

from scipy import optimize, special
import gurobipy as grb

from sklearn.preprocessing import LabelEncoder
from IPython.display import display, HTML

thePath = os.getcwd().split('code_py_mec_optim\\')[0]
travelmode = pd.read_csv(thePath + 'data_mec_optim\\demand_travelmode\\travelmodedata.csv', sep=',')

In [2]:
thePath = os.getcwd().split('Pycode_PC')[0]
travelmode = pd.read_csv(thePath + 'data_mec_optim\\demand_travelmode\\travelmodedata.csv', sep=',')

In [3]:
lb = LabelEncoder() 
travelmode['choice'] = lb.fit_transform(travelmode['choice'])
#travelmode['mode'] = lb.fit_transform(travelmode['mode'])

In [4]:
nobs = travelmode.shape[0]
ncols = travelmode.shape[1]
nbchoices = 4
ninds = int(nobs/nbchoices)

In [5]:
muhat_i_y = travelmode['choice'].values.reshape(ninds,nbchoices).T
muhat_iy = muhat_i_y.flatten()

In [6]:
sorter = ['air', 'train', 'bus', 'car']
travelmode['mode'] = travelmode['mode'].astype("category")
travelmode['mode'].cat.set_categories(sorter, inplace=True)
travelmode.columns = travelmode.columns.str.strip()
travelmode.sort_values(['mode','individual'], inplace = True)

In [7]:
travelmode.head()

,individual,mode,choice,wait,vcost,travel,gcost,income,size
0,1,air,0,69,59,100,70,35,1
4,2,air,0,64,58,68,68,30,2
8,3,air,0,69,115,125,129,40,1
12,4,air,0,64,49,68,59,70,3
16,5,air,0,64,60,144,82,45,2


In [8]:
Phi_iy_k = np.column_stack((np.kron(np.identity(4)[0:4,1:4],np.repeat(1, ninds).reshape(ninds,1)), - travelmode['travel'].values, - (travelmode['travel']*travelmode['income']).values, - travelmode['gcost'].values))

In [9]:
nbK = Phi_iy_k.shape[1]
phi_mean = Phi_iy_k.mean(axis = 0)
phi_stdev = Phi_iy_k.std(axis = 0, ddof = 1)
Phi_iy_k = ((Phi_iy_k - phi_mean).T/phi_stdev[:,None]).T

In [10]:
def log_likelihood(theta):
    nbK = np.asarray(theta).shape[0]
    Xtheta = Phi_iy_k.dot(theta)/sigma
    Xthetamat_iy = Xtheta.reshape(nbchoices, ninds).T
    max_i = np.amax(Xthetamat_iy, axis = 1)
    expPhi_iy = np.exp((Xthetamat_iy.T -max_i).T)
    d_i = np.sum(expPhi_iy, axis = 1)
    
    val = np.sum(np.multiply(Xtheta,muhat_iy))  - np.sum(max_i) - sigma * np.sum(np.log(d_i))

    return -val

In [11]:
def grad_log_likelihood(theta):
    nbK = np.asarray(theta).shape[0]
    Xtheta = Phi_iy_k.dot(theta)/sigma
    Xthetamat_iy = Xtheta.reshape(nbchoices, ninds).T
    max_i = np.amax(Xthetamat_iy, axis = 1)
    expPhi_iy = np.exp((Xthetamat_iy.T -max_i).T)
    d_i = np.sum(expPhi_iy, axis = 1)
    
    temp_mat = np.multiply(Phi_iy_k.T, expPhi_iy.T.flatten()).T
    list_temp = []
    for i in range(nbchoices):
        list_temp.append(temp_mat[i*ninds:(i+1)*ninds,])
    n_i_k = np.sum(list_temp,axis = 0)
    
    thegrad = muhat_iy.reshape(1,nbchoices*ninds).dot(Phi_iy_k).flatten() - np.sum(n_i_k.T/d_i, axis = 1)

    return -thegrad

In [12]:
theta0 = np.repeat(0,nbK)
sigma = 1
outcome = optimize.minimize(log_likelihood,method = 'CG',jac = grad_log_likelihood, x0 = theta0)

In [13]:
outcome

     fun: 264.66296167818143
     jac: array([ 6.40730590e-07, -1.57541966e-06, -4.43855805e-07,  3.02417753e-07,
        7.69659202e-07, -2.22239347e-07])
 message: 'Optimization terminated successfully.'
    nfev: 61
     nit: 32
    njev: 61
  status: 0
 success: True
       x: array([0.71120457, 0.36631106, 0.50859501, 0.41287861, 0.52396691,
       0.43816128])

In [14]:
temp_mle = 1 / outcome['x'][0]
theta_mle = outcome['x']*temp_mle

In [15]:
objList = [i for i in range(nbK+ninds)]
lenobj = len(objList)
c = np.concatenate((muhat_iy.reshape(1,nbchoices*ninds).dot(Phi_iy_k).flatten(),np.repeat(-1,ninds)))

m = grb.Model('lp')
m.ModelSense = -1
x = m.addVars(objList, obj = c, name='x', lb = -math.inf)

Academic license - for non-commercial use only


In [16]:
rhs = np.repeat(0,ninds*nbchoices)
id_ind = np.identity(ninds) 

for i in range(ninds*nbchoices):
    L = grb.LinExpr(np.concatenate((-Phi_iy_k[i,:],id_ind[i%210,:])),x.select('*'))
    m.addConstr(L,'>',rhs[i])
    
Last = grb.LinExpr(np.concatenate(([1],np.repeat(0,lenobj-1))),x.select('*'))
m.addConstr(Last, '=', 1)

<gurobi.Constr *Awaiting Model Update*>

In [17]:
m.optimize()

# Print the solution
if m.status == grb.GRB.Status.OPTIMAL:
    print("Value of the problem (Gurobi) =", m.objval)
    opt_x = m.getAttr('x',x).select('*')

Optimize a model with 841 rows, 216 columns and 5881 nonzeros
Coefficient statistics:
  Matrix range     [3e-03, 5e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 211 rows and 1 columns
Presolve time: 0.02s
Presolved: 630 rows, 215 columns, 2925 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     399   -5.2107549e+01   0.000000e+00   0.000000e+00      0s

Solved in 399 iterations and 0.05 seconds
Optimal objective -5.210754938e+01
Value of the problem (Gurobi) = -52.10754937546851


In [18]:
theta_lp = np.array(opt_x[:nbK])
indMax=100
tempMax=temp_mle
outcomemat = np.zeros((indMax+1,nbK-1))

In [19]:
def log_likelihood_fixedtemp(subsetoftheta, *temp):
    val = log_likelihood(np.append(1/temp[0],subsetoftheta))
    
    return val

In [20]:
def grad_log_likelihood_fixedtemp(subsetoftheta,*temp):
    val = np.delete(grad_log_likelihood(np.append(1/temp[0],subsetoftheta)),[0])
    
    return val

In [21]:
outcomemat[0,:] = np.delete(theta_lp,[0])
iterMax = indMax+1
for k in range(2,iterMax+1,1):
    thetemp = tempMax * (k-1)/indMax
    outcomeFixedTemp = optimize.minimize(log_likelihood_fixedtemp,method = 'CG',jac = grad_log_likelihood_fixedtemp, args = (thetemp,),  x0 = theta0[:-1])
    outcomemat[k-1,:] = outcomeFixedTemp['x']*thetemp

The zero-temperature estimator is:

In [22]:
print(outcomemat[1,:])

[ 1.03807212  0.97377372  1.01718642  0.18539829 -0.25345272]


The mle estimator is:

In [23]:
print(outcomemat[indMax,])

[0.51505719 0.71511769 0.58053423 0.73673156 0.61608345]


In [119]:
nbB = 50
thetemp = 1

In [120]:
objListnew = [i for i in range(ninds*nbB+nbK)]
lenobj = len(objListnew)

newc = np.concatenate((muhat_iy.reshape(1,nbchoices*ninds).dot(Phi_iy_k).flatten(),np.repeat(-1/nbB,ninds*nbB)))
newm = grb.Model('new_lp')
newm.ModelSense = -1
newx = newm.addVars(objListnew, obj = newc, name='newx', lb = -math.inf)

In [121]:
id_ind = np.identity(ninds*nbB) 
epsilon_biy = special.digamma(1) -np.log(-np.log(np.random.uniform(0,1,ninds*nbchoices*nbB)))

In [122]:
ptm = time.time()
for i in range(ninds*nbchoices*nbB):
    L = grb.LinExpr(np.concatenate((-Phi_iy_k[i//nbB,:],id_ind[i%(ninds*nbB),:])),newx.select('*'))
    newm.addConstr(L,'>',epsilon_biy[i])
    
#newm.addConstrs(grb.LinExpr([np.concatenate((-Phi_iy_k[i//nbB,:],id_ind[i%(ninds*nbB),:])) for i in range(ninds*nbchoices*nbB)],[newx.select('*') for i in range(ninds*nbchoices*nbB)])<epsilon_biy[i])
    
diff = time.time() - ptm
print('Time elapsed = ', diff, 's.')

Time elapsed =  381.37516736984253 s.


In [54]:
newm.optimize()

if newm.status == grb.GRB.Status.OPTIMAL:
    print("Value of the problem (Gurobi) =", newm.objval)
    opt_x = newm.getAttr('x',x).select('*')

Optimize a model with 8400 rows, 2106 columns and 58800 nonzeros
Coefficient statistics:
  Matrix range     [3e-03, 5e+00]
  Objective range  [1e-01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-05, 9e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.05s
Presolved: 2106 rows, 8400 columns, 58800 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.262e+04
 Factor NZ  : 1.510e+04 (roughly 4 MBytes of memory)
 Factor Ops : 1.152e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   9.64236972e+00 -2.49954931e+00  8.70e+00 7.17e+00  3.45e+01     0s
   1  -1.05258196e+02 -3.21659478e+03  1.41e-01 1.42e-14  9.13e-01     0s
   2  -6.52342387e+01 -1.09059886e+03  8.95e-13 7.99e-15  1.22e-01     0s
   3  -1.27382527e+02 -4.51921502e+02  1.12e-12 4.88e-15  3.86e-02     0s
   4 

In [67]:
newtheta_lp = np.asarray(opt_x[0:nbK-1])/opt_x[0]

The lp-simulated estimator is:

In [68]:
np.delete(newtheta_lp,[0])

array([0.59795177, 0.7560584 , 0.58530629, 0.8079241 ])